In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

# 데이터 불러오기

In [ ]:
# 데이터 불러오기 (train set)

# 데이터 위치는 맨 처음 커널을 생성했을때 작성되어있는 블록을 실행하면 나온다.
train=pd.read_csv("/kaggle/input/bike-sharing-demand/train.csv")
train.head()

In [ ]:
# 데이터 불러오기 (test set)
test=pd.read_csv("/kaggle/input/bike-sharing-demand/test.csv")
test.head()

# train set 전처리

In [ ]:
# train set 데이터 형태 확인하기

# 데이터 슬라이싱 : 데이터이름["열이름","열이름"...]
# pd.to_datetime으로 날짜변수로 변환하면 dt객체에 연도 정보는 year, 요일 정보는 weekday에 저장된다. 참고로 weekday중 Monday=0, Sunday=6 이다.
train["datetime"] = pd.to_datetime(train["datetime"])
train["dateyear"] = train["datetime"].dt.year
train["weekday"] = train["datetime"].dt.weekday
train["hour"] = train["datetime"].dt.hour


train.dtypes

In [ ]:
# 데이터 앞부분 확인
train.head()

In [ ]:
# 빈도분석
train["weekday"].value_counts()

In [ ]:
train["dateyear"].value_counts()

In [ ]:
# 요일별 자전거 대여량의 평균, 중앙값
print(train.groupby("weekday")["count"].mean())
print(train.groupby("weekday")["count"].median())

In [ ]:
train.groupby("holiday")["count"].mean()

In [ ]:
print(train["weekday"].value_counts())
train["holiday"].value_counts()

In [ ]:
# 
weekend_df = train[train["workingday"] == 0]
weekday_df = train[train["workingday"] == 1]

In [ ]:
train2 = train.drop(["datetime","casual","registered","count"],axis = 1)
train.head()
train2.head()

# test set 전처리

In [ ]:
# test set 데이터 형태 확인하기
test["datetime"] = pd.to_datetime(test["datetime"])
test.dtypes

In [ ]:
# datetime 컬럼 object 타입에서 datetime으로 변환

test["dateyear"] = test["datetime"].dt.year
test["weekday"] = test["datetime"].dt.weekday
# test["month"] = test["datetime"].dt.month
# test["day"] = test["datetime"].dt.day
test["hour"] = test["datetime"].dt.hour


test.head()

In [ ]:
test2 = test.drop(["datetime"],axis = 1)
test2.head()

# 데이터 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize = (10,6))
sns.boxplot(weekend_df["hour"],train["count"])

In [ ]:
# 정답값이 왼쪽으로 치우쳤을때 로그변환을 통해 정규분포에 가까운 형태로 변환해준다.
plt.figure(figsize = (10,6))
sns.distplot(train["count"])

In [ ]:
plt.figure(figsize = (10,6))
sns.distplot(np.log(train["count"]))

In [ ]:
plt.figure(figsize = (10,6))
sns.boxplot(weekday_df["hour"],train["count"])

# 모델링

In [ ]:
# from sklearn.ensemble import RandomForestRegressor  # 사이킷런 라이브러리의 랜덤포레스트 
# rf = RandomForestRegressor(n_estimators = 100)      # n_estimators : 트리의 갯수 지정
# rf.fit(train2,np.log(train["count"]))                       # train2 데이터에서 count열을 기준으로 모델 학습  
# result = rf.predict(test2)                          

In [ ]:
# from lightgbm import LGBMRegressor
# lgb = LGBMRegressor()
# lgb.fit(train2,np.log(train["count"]))                       # train2 데이터에서 count열을 기준으로 모델 학습  
# result = lgb.predict(test2)

In [ ]:
train2.head()

In [ ]:
test2.head()

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor(max_depth = 6)
xgb.fit(train2,np.log(train["count"]))                         
result = xgb.predict(test2)

In [ ]:
sub = pd.read_csv("/kaggle/input/bike-sharing-demand/sampleSubmission.csv")
sub.head()

In [ ]:
sub["count"] = np.exp(result)
sub.head()

In [ ]:
sub.to_csv("sub.csv",index = False)

In [ ]:
pd.Series(xgb.feature_importances_,index = train2.columns).sort_values(ascending = False)

In [ ]:
train.head()

In [ ]:
train["month"] = train["datetime"].dt.month

In [ ]:
plt.figure(figsize = (10,6))
sns.barplot(train["month"],train["count"])